# Functions
As you might have guess, functions are really important in a functional language.  Functions are where the real work is done in a program.  A function in F# takes some data as input and creates some new data a output.  A really important concept in functional programming is that you can *combine* or *compose* functions to create new functions.

In F# a function is a value just like any other value, so you define a function with the `let` keyword.  A function is defined by its name, arguments (input) and ouput type.  In other programming languages you ususally see arguments surrounded by parentheses and commas, but not in  F#.

Let us define our first function.

In [35]:
let square x =
    x * x
    
square 4

16

The `square` function takes one argument `x`.  Since we call `square` with an integer, F# will assume that `x` is an integer and the output value is also an integer.

You can define functions with multiple arguments.

In [36]:
let add x y = 
    x + y

You can also define functions that don't have a name with the `fun` keyword.  You have already used this in the example with `List.map`.

In [37]:
type PersonPremium = 
    {
        PersonId: string;
        Premium: decimal;
    }
    
let data = 
    [
        { PersonId = "001"; Premium = 100m };
        { PersonId = "002"; Premium = 200m };
    ]
    
let totalPremium =
    data 
    |> List.map (fun p -> p.Premium)
    |> List.sum
    
totalPremium

300

## The Pipe Operator
This brings us to the pipe operator `|>`.  By now, you have seen it used a couple of times.  The pipe operator replaces the last argument of the function on the left hand side with whatever is on the right hand side.  So lines 1 and 2 below are equivalent:

In [38]:
4 |> square
square 4

16

Similarly, this:

In [39]:
let totalpremium =
    data 
    |> List.map (fun p -> p.Premium)
    |> List.sum
totalPremium

300

... is equivalent to this:

In [40]:
let listOfPremiums = List.map (fun p -> p.Premium) data
let totalPremium = List.sum listOfPremiums
totalPremium

300

You might see why most people think the `|>` operator is very elegant.

## Composition

In mathematics you can compose functions $f: A \mapsto B$, $g: B \mapsto C$ as 

$$h = g \circ f: A \mapsto C.$$

You can also put it as $h(X) = g(f(X))$.

You can do the same in F# with the `>>` operator, just note that the operands are switched around, so that you would use `f >> g`.  Let's say you want to calculate a yearly fee for all your monthly premiums.  First you create a function `f` the calculates the yearly premium and then a function `g` that calculates the fee of 3 percent.  You want to combine those two to one function `h`.

In [52]:

type MonthlyPremium = MontlyPremium of decimal
type AnnualPremium = AnnualPremium of decimal

let f (MontlyPremium premium) = 12m * premium |> AnnualPremium
let g (AnnualPremium premium) = 0.03m * premium

let h = f >> g // In real life you would use better function names, right?
let annualFee = h (MontlyPremium 10m)
yearlyFee

3.60

Note how we use single case discriminated unions to distinguish between monthly and annual premiums.

There are other more advanced ways to compose functions in F# which we will get back to later. 

## Partial Application
You can define new functions from existing functions by specifying some of the arguments (starting from the left) but not all.  This is called *partial application*.  Let us redefine the function above for calculating the fee to show how it is done.

In [55]:
let calculateFee fee (AnnualPremium premium) =
    fee * premium
    
calculateFee 0.03m (AnnualPremium 120m)

3.60

If the fee is always 3 percent throughout your program, it may get tiring to always type it out whenever you call `calculateFee`.  So using partial application you can define a new function with the fee set to 3 percent.

In [59]:
let calculateFee3Percent = 
    calculateFee 0.03m // Only specify the first argument
    
calculateFee3Percent (AnnualPremium 120m)

3.60

The new function `calculateFee3Percent` is a function of 1 argument.

Partial application may help you understand how the pipe operator works.